In [1]:
%load_ext autoreload
%autoreload 2

## initialization

In [2]:
from beam_search import create_dataset
from data import encode_line, load_vocabulary
from model import Transformer
import torch
import torch.nn as nn
import argparse

args = argparse.Namespace()
args.src_vocab = '../corpus/vocab_en_fr.txt'
args.tgt_vocab = '../corpus/vocab_en_fr.txt'
args.ckpt = './averaged_checkpoint.pt'
args.device = 'cpu'

batch_size = 16
beam_size = 5
length_penalty = 1
max_length = 256

source_vocabulary, _ = load_vocabulary(args.src_vocab)
target_vocabulary, target_vocabulary_rev = load_vocabulary(args.tgt_vocab)

bos = target_vocabulary["<s>"]
eos = target_vocabulary["</s>"]
model = Transformer(
    len(source_vocabulary),
    len(target_vocabulary),
    share_embeddings=True,
)
checkpoint = torch.load(args.ckpt)
model.load_state_dict(checkpoint["model"])
model.to(args.device)
model.eval()
n = 0
for param in model.parameters():
    # print(param.shape)
    n += param.numel()
print(n)

source_path = '../corpus/rep_test.en.tok'
dataset = create_dataset(source_path, source_vocabulary, args.device)
ref_path = '../corpus/rep_test.fr'
with open(ref_path, 'r', encoding='utf-8') as file:
    ref = file.readlines()
ref[0]

209129472


'Cette fois-ci, la baisse est due à la chute des actions au Wall Street.\n'

## helper functions

In [3]:
def view(hypotheses):
    for hypo in hypotheses:
        score = hypo[0].item()
        tokens = hypo[1]
        if tokens and tokens[-1] == eos:
            tokens.pop(-1)
        tokens = [target_vocabulary_rev[token_id] for token_id in tokens]
        print(f"{score:.4f}", " ".join(tokens))

## inference analysis

In [4]:
from my_beam_search import beam_search

In [5]:
batch = next(iter(dataset))

In [6]:
from utils import *

## original version

In [36]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=False)

my beam search
None


 21%|██        | 53/256 [08:11<31:22,  9.27s/it]   


result is a batch of data, whose element is (score, sentence)

In [9]:
view(result[0])

-0.2325 Cette fois ￭, la chute des actions de Wall Street est responsable de la chute ￭.
-0.2389 Cette fois ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2402 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est responsable de la chute ￭.
-0.2528 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2846 Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3051 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3260 Cette fois ￭, la chute des actions de Wall Street est responsable de la chute ￭. cours ￭.


Maybe what we want is 

`Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse ￭.`

However, it is not the second candidate

## naive version

In [8]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    penalized_result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=True, naive_penalty=5)

my beam search
5


  0%|          | 0/256 [02:04<?, ?it/s]


AssertionError: 

In [55]:
penalty, penalty2 = penalized_result

In [56]:
penalty.nonzero().equal(penalty2.nonzero())

False

In [61]:
set(tuple(idx.tolist()) for idx in penalty.nonzero()).difference(
    set(tuple(idx.tolist()) for idx in penalty2.nonzero()))

{(1, 5), (1, 26), (1, 2206), (3, 5), (3, 26), (3, 2206), (3, 2948)}

In [64]:
from utils import *
diff(penalty, penalty2)

{(1, 5), (1, 26), (1, 2206), (3, 5), (3, 26), (3, 2206), (3, 2948)}

In [11]:
view(penalized_result[0])

-0.3882 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est responsable ￭.
-0.4981 Cette fois ￭-￭ ci ￭, la baisse des actions de Wall Street en est responsable ￭.
-0.5306 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭.
-0.7309 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de cette baisse ￭.
-0.7371 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭'￭ origine de cette baisse ￭.
-0.8594 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de la baisse ￭.
-0.9045 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de cette diminution ￭.
-0.9075 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭'￭ origine de cette diminution ￭.


## mask version

In [37]:
from my_beam_search import calc_mask


mask = calc_mask(target_vocabulary)

['ADP', 'AUX', 'CCONJ', 'DET', 'NUM', 'PART', 'PRON', 'SCONJ', 'PUNCT', 'SYM', 'X']


100%|██████████| 31996/31996 [03:20<00:00, 159.38it/s]


In [41]:
torch.save(mask, 'penalty_mask.pt')

In [12]:
mask = torch.load('penalty_mask.pt')

In [32]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    penalized_result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=True, naive_penalty=5, mask=mask)

my beam search
5


 21%|██        | 54/256 [38:02<2:22:19, 42.27s/it]   


In [14]:
view(penalized_result[0])

-0.2389 Cette fois ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2528 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2846 Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3051 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3103 Cette fois ￭, la baisse des actions de Wall Street est responsable de la chute ￭.
-0.3845 Cette fois ￭, la chute des actions de Wall Street en est la cause ￭. des cours ￭.
-0.4720 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭. ￭.
